In [37]:
import requests
from urllib.parse import urlencode, urlparse, parse_qsl

In [38]:
GOOGLE_API_KEY=''

In [39]:
class GoogleMapClient(object):
    lat = None
    lng = None
    data_type = 'json'
    location_query = None
    api_key = None
    
    def __init__(self, api_key=None, address_or_postcode=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if api_key == None:
            raise Exception('API key is required')
        self.api_key = api_key
        self.location_query = address_or_postcode
        if self.location_query != None:
            self.extract_lat_lng()
        
    
    def extract_lat_lng(self, location=None):
        loc_query = self.location_query
        if location != None:
            loc_query = location
        endpiont= f'https://maps.googleapis.com/maps/api/geocode/{self.data_type}'
        params= {'address': loc_query, 'key': self.api_key}
        url_params = urlencode(params)
        url = f'{endpiont}?{url_params}'
        r = requests.get(url)
        if r.status_code not in range(200, 299):
            return {}
        latlong = {}
        try:
            latlong = r.json()['results'][0]['geometry']['location']
        except:
            pass
        lat, lng = latlong.get('lat'), latlong.get('lng')
        self.lat = lat
        self.lng = lng
        return lat, lng
    
    
    def search(self, keyword="Doner Kebab", redius=1000, location=None):
        lat, lng = self.lat, self.lng
        if location != None:
            lat, lng = self.extract_lat_lng()
        endpoint = f"https://maps.googleapis.com/maps/api/place/nearbysearch/{self.data_type}"
        params = {
            "key": self.api_key,
            "location": f"{lat},{lng}",
            "radius": redius,
            "keyword": "Doner Kebab"
        }
        params_encoded = urlencode(params)
        places_url = f"{endpoint}?{params_encoded}"

        r = requests.get(places_url)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def detail(self, place_id="ChIJlXOKcDC3j4ARzal-5j-p-FY", fields=["name", "rating", "formatted_phone_number", "formatted_address"]):
        detail_base_endpoint = f"https://maps.googleapis.com/maps/api/place/details/{self.data_type}"
        detail_params = {
            "place_id": f"{place_id}",
            "fields" : ",".join(fields),
            "key": self.api_key
        }
        detail_params_encoded = urlencode(detail_params)
        detail_url = f"{detail_base_endpoint}?{detail_params_encoded}"
        r = requests.get(detail_url)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

In [40]:
client = GoogleMapClient(api_key=GOOGLE_API_KEY, address_or_postcode='Portobello, UK')
print(client.lat, client.lng)

55.9496024 -3.1092874


In [43]:
client2 = GoogleMapClient(api_key=GOOGLE_API_KEY, address_or_postcode='EH8 9UG')
print(client2.lat, client2.lng)

55.9492382 -3.1782842


In [42]:
client.detail(place_id='ChIJNQsn1Kq5h0gRSQnhPRkr9J4')

{'html_attributions': [],
 'result': {'formatted_address': '6a Milton Rd W, Edinburgh EH15 1LF, UK',
  'formatted_phone_number': '0131 669 3219',
  'name': 'Milton Fry',
  'rating': 4.1},
 'status': 'OK'}

In [47]:
client.search('pizza')

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 55.9544229, 'lng': -3.1166498},
    'viewport': {'northeast': {'lat': 55.95574182989272,
      'lng': -3.115358370107277},
     'southwest': {'lat': 55.95304217010728, 'lng': -3.118058029892722}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
   'icon_background_color': '#FF9E67',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet',
   'name': 'Istanbul Takeaway',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 2988,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/114576900135953900755">Omer Rizwan</a>'],
     'photo_reference': 'Aap_uECbefLrut1siuwaQrdgIp8hF4XqvufgWnp0qg6KTQQZdK_g91_h-9xz2uNp7l1dvPIlu_3vM8lpOmr7vcSDB7k_Ad6BvDFNBDCvqZmbu-fuQdTuTb-eR99mHSQhq-t4mAG5V11YhYiaS50qfevymYxaNx8OwBWZe6KGiAfTaljtxLlA',
     'width': 5312}],
   'place_id'

In [41]:
client.search('pizza', location='Edinburgh, UK')

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 55.9544229, 'lng': -3.1166498},
    'viewport': {'northeast': {'lat': 55.95574182989272,
      'lng': -3.115358370107277},
     'southwest': {'lat': 55.95304217010728, 'lng': -3.118058029892722}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
   'icon_background_color': '#FF9E67',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet',
   'name': 'Istanbul Takeaway',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 2988,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/114576900135953900755">Omer Rizwan</a>'],
     'photo_reference': 'Aap_uECYntWaj7dMqcOcH3VKc6Ork1QryU7zPvN3ZvhqFJ-2WlJn82lZaewHSaWQbIu8KTx_gjCCODOI-hBQt4LkR8keB5yxigyLOfS6Oof305fvjlDI3J6fuFinm1R0r8irKhsaShUehMjNbJepfzZ_6Yee7PZk1kZXS51jAObJ27AXaRdX',
     'width': 5312}],
   'place_id'